# StyleGAN2 with Augmentations

This notebook shows code for training with image augmentations. For more info on this technique see [Training Generative Adversarial Networks with Limited Data](https://arxiv.org/abs/2006.06676). 

This code comes from [Sid Black](https://github.com/sdtblck/stylegan2). A huge thanks to him for doing all of this work 🙏

If this is your first time using StyleGAN2 on Colab I recommend watching some of my YouTube videos first. Start with [this one](https://www.youtube.com/watch?v=hv3A62Ojqdg). A video on this notebook and technique is [here](https://youtu.be/D3a9DFykfxI)

In [2]:
#always use tensorflow1

%tensorflow_version 1.x


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Initial Setup

Run this cell if you’ve never run this repo in your Drive account. SKIP it if you have.

In [ ]:
#SKIP this if you already have a stylegan2 folder in your google drive
%cd /content/drive/MyDrive/
!mkdir stylegan2-aug-colab
%cd stylegan2-aug-colab/
!git clone -b augs-attn https://github.com/dvschultz/stylegan2
%cd stylegan2
!mkdir pkl
%cd pkl
!gdown --id 1JLqXE5bGZnlu2BkbLPD5_ZxoO3Nz-AvF #inception: https://drive.google.com/open?id=1JLqXE5bGZnlu2BkbLPD5_ZxoO3Nz-AvF
%cd ../
!mkdir results
!mkdir results/00001-pretrained
%cd results/00001-pretrained
!gdown --id 1UlDmJVLLnBD9SnLSMXeiZRO6g-OMQCA_
%cd ../../
%mkdir datasets

/content/drive/MyDrive
mkdir: cannot create directory ‘stylegan2-aug-colab’: File exists
/content/drive/MyDrive/stylegan2-aug-colab
fatal: destination path 'stylegan2' already exists and is not an empty directory.
/content/drive/MyDrive/stylegan2-aug-colab/stylegan2
mkdir: cannot create directory ‘pkl’: File exists
/content/drive/MyDrive/stylegan2-aug-colab/stylegan2/pkl
Downloading...
From: https://drive.google.com/uc?id=1JLqXE5bGZnlu2BkbLPD5_ZxoO3Nz-AvF
To: /content/drive/MyDrive/stylegan2-aug-colab/stylegan2/pkl/inception_v3_features.pkl
87.3MB [00:01, 63.9MB/s]
/content/drive/My Drive/stylegan2-aug-colab/stylegan2
mkdir: cannot create directory ‘results’: File exists
mkdir: cannot create directory ‘results/00001-pretrained’: File exists
/content/drive/My Drive/stylegan2-aug-colab/stylegan2/results/00001-pretrained
Downloading...
From: https://drive.google.com/uc?id=1UlDmJVLLnBD9SnLSMXeiZRO6g-OMQCA_
To: /content/drive/My Drive/stylegan2-aug-colab/stylegan2/results/00001-pretrained/s

## Return Setup
Run this cell if you’re picking up from a previous training.

In [6]:
%cd /content/drive/MyDrive/stylegan2-aug-colab/stylegan2
#!git pull

/content/drive/MyDrive/stylegan2-aug-colab/stylegan2


## Training

In the next cell set your pkl and `resume_kimg` counter. If this is your first time the settings below should work for you.

In [7]:
pkl = "/content/drive/MyDrive/stylegan2-aug-colab/stylegan2/results/00074-stylegan2-SoomeenHahm-1gpu-config-f/network-snapshot-012016.pkl"
resume_kimg = 12016

if you need to make a dataset run below


The first address is the destination tf Records file

In [ ]:
!python dataset_tool.py create_from_images_raw /content/drive/MyDrive/stylegan2-aug-colab/stylegan2/datasets/SoomeenHahm /content/drive/MyDrive/stylegan2-aug-colab/stylegan2/SoomeenImages

Loading images from "/content/drive/MyDrive/stylegan2-aug-colab/stylegan2/SoomeenImages"
detected 188 images ...
Shuffle the images...
Creating dataset "/content/drive/MyDrive/stylegan2-aug-colab/stylegan2/datasets/SoomeenHahm"
Adding the images to tfrecords ...
added images 0
Added 188 images.


Most of the settings match the skyflynil or pbaylies fork of StyleGAN2. The big difference here is the `AUG_PROB` environment setting. This tells the training loop how often to modify the real and fake images with augmentations. The default is `0.1` If you have a small training set you may want to go higher than that but note that the Karras paper does say if you set this value too high you may find it bleeds into the outputs.

In [ ]:
!AUG_PROB=0.2 python run_training.py --num-gpus=1 --mirror-augment=True --data-dir=./datasets --dataset=Beast --config=config-f --resume-pkl=$pkl --resume-kimg=$resume_kimg --augmentations=True --metrics=None

Local submit - run_dir: results/00034-stylegan2-Beast-1gpu-config-f
dnnlib: Running training.training_loop.training_loop() on localhost...
Streaming data using training.dataset.TFRecordDataset...
Dataset shape = [3, 512, 512]
Dynamic range = [0, 255]
Label size    = 0
Loading networks from "/content/drive/MyDrive/stylegan2-aug-colab/stylegan2/results/00033-stylegan2-Beast-1gpu-config-f/network-snapshot-013837.pkl"...
Setting up TensorFlow plugin "fused_bias_act.cu": Preprocessing... Loading... Done.
Setting up TensorFlow plugin "upfirdn_2d.cu": Preprocessing... Loading... Done.

G                             Params    OutputShape         WeightShape     
---                           ---       ---                 ---             
latents_in                    -         (?, 512)            -               
labels_in                     -         (?, 0)              -               
lod                           -         ()                  -               
dlatent_avg                  

# Projection
Once the network is trained we can project in selected images.  The first step is to create a dataset file for those images.


In [ ]:
!python dataset_tool.py create_from_images_raw /content/drive/MyDrive/stylegan2-aug-colab/stylegan2/datasets/ProjectFiles /content/drive/MyDrive/stylegan2-aug-colab/stylegan2/projectMe

Loading images from "/content/drive/MyDrive/stylegan2-aug-colab/stylegan2/projectMe"
detected 45 images ...
Shuffle the images...
Creating dataset "/content/drive/MyDrive/stylegan2-aug-colab/stylegan2/datasets/ProjectFiles"
Adding the images to tfrecords ...
added images 0
Added 45 images.


Once the new dataset for projection is created you can run this to produce the outputs.


1.   set the network flag to the location of your most recent output .pkl
2.   check your dataset name and dir



In [ ]:
!python run_projector.py project-real-images --network=/content/drive/MyDrive/stylegan2-aug-colab/stylegan2/results/00007-stylegan2-Ginza-1gpu-config-f/network-snapshot-010054.pkl --dataset=ProjectFiles --data-dir=/content/drive/MyDrive/stylegan2-aug-colab/stylegan2/datasets --num-images=45

Local submit - run_dir: results/00010-project-real-images
dnnlib: Running run_projector.project_real_images() on localhost...
Loading networks from "/content/drive/MyDrive/stylegan2-aug-colab/stylegan2/results/00007-stylegan2-Ginza-1gpu-config-f/network-snapshot-010054.pkl"...
Setting up TensorFlow plugin "fused_bias_act.cu": Preprocessing... Loading... Done.
Setting up TensorFlow plugin "upfirdn_2d.cu": Preprocessing... Loading... Done.
Loading images from "ProjectFiles"...
Projecting image 0/45 ...
Projecting image 1/45 ...
Projecting image 2/45 ...
Projecting image 3/45 ...
Projecting image 4/45 ...
Projecting image 5/45 ...
Projecting image 6/45 ...
Projecting image 7/45 ...
Projecting image 8/45 ...
Projecting image 9/45 ...
Projecting image 10/45 ...
Projecting image 11/45 ...
Projecting image 12/45 ...
Projecting image 13/45 ...
Projecting image 14/45 ...
Projecting image 15/45 ...
Projecting image 16/45 ...
Projecting image 17/45 ...
Projecting image 18/45 ...
Projecting image 

# RandomImage Generation and Interpolation

In [9]:
%cd /content/drive/MyDrive/stylegan2-aug-colab/stylegan2

/content/drive/MyDrive/stylegan2-aug-colab/stylegan2


This code will create a sequence of png's smoothly blending between random points in the network.   

1.   Change line 41 to your network
2.   Change Tokyo to your image name on line 83



In [11]:
# Copyright (c) 2019, NVIDIA Corporation. All rights reserved.
#
# This work is made available under the Nvidia Source Code License-NC.
# To view a copy of this license, visit
# https://nvlabs.github.io/stylegan2/license.html

import argparse
import numpy as np
import PIL.Image
import dnnlib
import dnnlib.tflib as tflib


import pretrained_networks

#----------------------------------------------------------------------------
def create_image_grid(images, grid_size=None):
    assert images.ndim == 3 or images.ndim == 4
    num, img_h, img_w, channels = images.shape

    if grid_size is not None:
        grid_w, grid_h = tuple(grid_size)
    else:
        grid_w = max(int(np.ceil(np.sqrt(num))), 1)
        grid_h = max((num - 1) // grid_w + 1, 1)

    grid = np.zeros([grid_h * img_h, grid_w * img_w, channels], dtype=images.dtype)
    for idx in range(num):
        x = (idx % grid_w) * img_w
        y = (idx // grid_w) * img_h
        grid[y: y + img_h, x: x + img_w] = images[idx]
    return grid

def genGrid():

    tflib.init_tf()

    # Load pre-trained network.

    ## NOTE: insert model here:
    _G, _D, Gs = pretrained_networks.load_networks("network-snapshot-012016.pkl")


    grid_size = [8,5]

    image_zoom = 1
    duration_sec = 60
    smoothing_sec = .5
    mp4_fps = 24
    random_seed = 8004
    minibatch_size = 8
    cnt = 0
    num_frames = int(np.rint(duration_sec * mp4_fps))
    random_state = np.random.RandomState(random_seed)

    # Generate latent vectors
    shape = [num_frames, np.prod(grid_size)] + Gs.input_shape[1:] # [frame, image, channel, component]
    all_latents = random_state.randn(*shape).astype(np.float32)
    import scipy
    all_latents = scipy.ndimage.gaussian_filter(all_latents, [smoothing_sec * mp4_fps] + [0] * len(Gs.input_shape), mode='wrap')
    all_latents /= np.sqrt(np.mean(np.square(all_latents)))


    # Frame generation func for moviepy.
    for t in range(num_frames):
        v = t/24
        frame_idx = int(np.clip(np.round(v * mp4_fps), 0, num_frames - 1))
        latents = all_latents[frame_idx]
        fmt = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)
        images = Gs.run(latents, None, truncation_psi=0.75,
                              randomize_noise=False, output_transform=fmt)

        grid = create_image_grid(images, grid_size)
        if image_zoom > 1:
            grid = scipy.ndimage.zoom(grid, [image_zoom, image_zoom, 1], order=0)
        if grid.shape[2] == 1:
            grid = grid.repeat(3, 2) # grayscale => RGB
        canvas = PIL.Image.fromarray(grid)
        cnt = t
        #cnt = int(t*24)
        cnt = "{:04d}".format(cnt)
        print(cnt)
        fName = 'images/BeastTotal_' + str(cnt) + '.png'


        canvas.save(fName)
        #return grid




def main():
   genGrid()

#----------------------------------------------------------------------------

if __name__ == "__main__":
    main()

#----------------------------------------------------------------------------


FileNotFoundError: ignored

# **StyleMixing Grid Production**

In [12]:
# Copyright (c) 2019, NVIDIA CORPORATION. All rights reserved.
#
# This work is licensed under the Creative Commons Attribution-NonCommercial
# 4.0 International License. To view a copy of this license, visit
# http://creativecommons.org/licenses/by-nc/4.0/ or send a letter to
# Creative Commons, PO Box 1866, Mountain View, CA 94042, USA.

"""Minimal script for reproducing the figures of the StyleGAN paper using pre-trained generators."""

import os
import pickle
import numpy as np
import PIL.Image
import dnnlib
import dnnlib.tflib as tflib


#----------------------------------------------------------------------------
# Helpers for loading and using pre-trained generators.



synthesis_kwargs = dict(output_transform=dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True), minibatch_size=8)

_Gs_cache = dict()

# def load_Gs(url):
#     if url not in _Gs_cache:
#         with dnnlib.util.open_url(url, cache_dir=config.cache_dir) as f:
#             _G, _D, Gs = pickle.load(f)
#         _Gs_cache[url] = Gs
#     return _Gs_cache[url]
def load_Gs():
    _G, _D, Gs = pickle.load(open("network-snapshot-012016.pkl","rb"))
    return Gs
#----------------------------------------------------------------------------
# Figures 2, 3, 10, 11, 12: Multi-resolution grid of uncurated result images.

def draw_uncurated_result_figure(png, Gs, cx, cy, cw, ch, rows, lods, seed):
    print(png)
    latents = np.random.RandomState(seed).randn(sum(rows * 2**lod for lod in lods), Gs.input_shape[1])
    images = Gs.run(latents, None, **synthesis_kwargs) # [seed, y, x, rgb]

    canvas = PIL.Image.new('RGB', (sum(cw // 2**lod for lod in lods), ch * rows), 'white')
    image_iter = iter(list(images))
    for col, lod in enumerate(lods):
        for row in range(rows * 2**lod):
            image = PIL.Image.fromarray(next(image_iter), 'RGB')
            image = image.crop((cx, cy, cx + cw, cy + ch))
            image = image.resize((cw // 2**lod, ch // 2**lod), PIL.Image.ANTIALIAS)
            canvas.paste(image, (sum(cw // 2**lod for lod in lods[:col]), row * ch // 2**lod))
    canvas.save(png)

#----------------------------------------------------------------------------
# Figure 3: Style mixing.

def draw_style_mixing_figure(png, Gs, w, h, src_seeds, dst_seeds, style_ranges):
    print(png)
    src_latents = np.stack(np.random.RandomState(seed).randn(Gs.input_shape[1]) for seed in src_seeds)
    dst_latents = np.stack(np.random.RandomState(seed).randn(Gs.input_shape[1]) for seed in dst_seeds)
    src_dlatents = Gs.components.mapping.run(src_latents, None) # [seed, layer, component]
    dst_dlatents = Gs.components.mapping.run(dst_latents, None) # [seed, layer, component]
    src_images = Gs.components.synthesis.run(src_dlatents, randomize_noise=False, **synthesis_kwargs)
    dst_images = Gs.components.synthesis.run(dst_dlatents, randomize_noise=False, **synthesis_kwargs)

    canvas = PIL.Image.new('RGB', (w * (len(src_seeds) + 1), h * (len(dst_seeds) + 1)), 'white')
    for col, src_image in enumerate(list(src_images)):
        canvas.paste(PIL.Image.fromarray(src_image, 'RGB'), ((col + 1) * w, 0))
    for row, dst_image in enumerate(list(dst_images)):
        canvas.paste(PIL.Image.fromarray(dst_image, 'RGB'), (0, (row + 1) * h))
        row_dlatents = np.stack([dst_dlatents[row]] * len(src_seeds))
        row_dlatents[:, style_ranges[row]] = src_dlatents[:, style_ranges[row]]
        row_images = Gs.components.synthesis.run(row_dlatents, randomize_noise=False, **synthesis_kwargs)
        for col, image in enumerate(list(row_images)):
            canvas.paste(PIL.Image.fromarray(image, 'RGB'), ((col + 1) * w, (row + 1) * h))
    canvas.save(png)

#----------------------------------------------------------------------------
# Figure 4: Noise detail.

def draw_noise_detail_figure(png, Gs, w, h, num_samples, seeds):
    print(png)
    canvas = PIL.Image.new('RGB', (w * 3, h * len(seeds)), 'white')
    for row, seed in enumerate(seeds):
        latents = np.stack([np.random.RandomState(seed).randn(Gs.input_shape[1])] * num_samples)
        images = Gs.run(latents, None, truncation_psi=1, **synthesis_kwargs)
        canvas.paste(PIL.Image.fromarray(images[0], 'RGB'), (0, row * h))
        for i in range(4):
            crop = PIL.Image.fromarray(images[i + 1], 'RGB')
            crop = crop.crop((650, 180, 906, 436))
            crop = crop.resize((w//2, h//2), PIL.Image.NEAREST)
            canvas.paste(crop, (w + (i%2) * w//2, row * h + (i//2) * h//2))
        diff = np.std(np.mean(images, axis=3), axis=0) * 4
        diff = np.clip(diff + 0.5, 0, 255).astype(np.uint8)
        canvas.paste(PIL.Image.fromarray(diff, 'L'), (w * 2, row * h))
    canvas.save(png)

#----------------------------------------------------------------------------
# Figure 5: Noise components.

def draw_noise_components_figure(png, Gs, w, h, seeds, noise_ranges, flips):
    print(png)
    Gsc = Gs.clone()
    noise_vars = [var for name, var in Gsc.components.synthesis.vars.items() if name.startswith('noise')]
    noise_pairs = list(zip(noise_vars, tflib.run(noise_vars))) # [(var, val), ...]
    latents = np.stack(np.random.RandomState(seed).randn(Gs.input_shape[1]) for seed in seeds)
    all_images = []
    for noise_range in noise_ranges:
        tflib.set_vars({var: val * (1 if i in noise_range else 0) for i, (var, val) in enumerate(noise_pairs)})
        range_images = Gsc.run(latents, None, truncation_psi=1, randomize_noise=False, **synthesis_kwargs)
        range_images[flips, :, :] = range_images[flips, :, ::-1]
        all_images.append(list(range_images))

    canvas = PIL.Image.new('RGB', (w * 2, h * 2), 'white')
    for col, col_images in enumerate(zip(*all_images)):
        canvas.paste(PIL.Image.fromarray(col_images[0], 'RGB').crop((0, 0, w//2, h)), (col * w, 0))
        canvas.paste(PIL.Image.fromarray(col_images[1], 'RGB').crop((w//2, 0, w, h)), (col * w + w//2, 0))
        canvas.paste(PIL.Image.fromarray(col_images[2], 'RGB').crop((0, 0, w//2, h)), (col * w, h))
        canvas.paste(PIL.Image.fromarray(col_images[3], 'RGB').crop((w//2, 0, w, h)), (col * w + w//2, h))
    canvas.save(png)

#----------------------------------------------------------------------------
# Figure 8: Truncation trick.

def draw_truncation_trick_figure(png, Gs, w, h, seeds, psis):
    print(png)
    latents = np.stack(np.random.RandomState(seed).randn(Gs.input_shape[1]) for seed in seeds)
    dlatents = Gs.components.mapping.run(latents, None) # [seed, layer, component]
    dlatent_avg = Gs.get_var('dlatent_avg') # [component]

    canvas = PIL.Image.new('RGB', (w * len(psis), h * len(seeds)), 'white')
    for row, dlatent in enumerate(list(dlatents)):
        row_dlatents = (dlatent[np.newaxis] - dlatent_avg) * np.reshape(psis, [-1, 1, 1]) + dlatent_avg
        row_images = Gs.components.synthesis.run(row_dlatents, randomize_noise=False, **synthesis_kwargs)
        for col, image in enumerate(list(row_images)):
            canvas.paste(PIL.Image.fromarray(image, 'RGB'), (col * w, row * h))
    canvas.save(png)

#----------------------------------------------------------------------------
# Main program.

def main():
    tflib.init_tf()
    DIR = "StyleMix"
    os.makedirs(DIR, exist_ok=True)

    startSeed = 800
    startDis = 200
    startPsi = 1
    quant = 16
    seedList = []

    distList = []
    psiList = []

    dimStep = 2/quant


    for i in range(quant):
        tempSeed = startSeed+ i
        tempDis = startDis +i
        seedList.append(tempSeed)
        if(i > 0):
            distList.append(tempDis)
        if(i%2 == 0):
            tempPsi = startPsi - (i * dimStep)
            psiList.append(tempPsi)

    
    draw_style_mixing_figure(os.path.join(DIR, 'BeastMix2.png'), load_Gs(), w=512, h=512, src_seeds=distList, dst_seeds=seedList, style_ranges=[range(0,4)]*16+[range(4,8)]*15+[range(8,16)])
    

#----------------------------------------------------------------------------

if __name__ == "__main__":
    main()

#----------------------------------------------------------------------------


FileNotFoundError: ignored

Quick Linear Interpolation

In [ ]:
import os
import pickle
import numpy as np
import PIL.Image
import dnnlib
import dnnlib.tflib as tflib


def main():
    DIR = "Blend"
    os.makedirs(DIR, exist_ok=True)
    network_pkl = "BeastTotal-013891.pkl"
    _G, _D, Gs = pretrained_networks.load_networks(network_pkl)
    rnd = np.random
    fmt = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)
    latent1 = rnd.randn(1,Gs.input_shape[1])
    latent2 = rnd.randn(1,Gs.input_shape[1])
    #loop through 10 times and out put interpolated images
    for i in range(10):
        factor = float(i)/10
        l1 = latent1*factor
        l2 = latent2*(1-factor)
        combinedLatent = l1+l2
        current_image = images = Gs.run(combinedLatent,None, truncation_psi=1.0,randomize_noise=False,output_transform=fmt)[0]
        image = PIL.Image.fromarray(current_image,'RGB')
        fName = DIR + "/"+ "beast_" + str(i) + ".png"
        image.save(fName)







if __name__ == "__main__":
    main()